Problem 1

In [3]:
#a
import numpy as np
import pandas as pd

data = pd.read_csv("Auto.csv", na_values="?")
data.dropna(inplace=True)

In [6]:
data["mpg_high"] = np.where(data["mpg"] > data["mpg"].median(), 1, 0)
data["orgn1"] = data["origin"] == 1
data["orgn2"] = data["origin"] == 2
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,orgn1,orgn2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,True,False
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,True,False
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,True,False
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,True,False
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,True,False


In [8]:
data['orgn1'] = data['orgn1'].astype('int32')
data['orgn2'] = data['orgn2'].astype('int32')
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,orgn1,orgn2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1,0


In [9]:
X = data[["cylinders", "displacement", "horsepower", "weight", "acceleration", "year", "orgn1", "orgn2"]]
y = data["mpg_high"]

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import timeit

seeds = np.random.choice(1000, 100, replace=False)
mse_list = []

start_time = timeit.default_timer()
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.65, random_state = seeds[i])
    logreg = LogisticRegression(solver="lbfgs", max_iter = 2000, n_jobs = None)
    results = logreg.fit(X_train, y_train)
    y_pred = results.predict(X_test)
    mse = ((y_test - y_pred) ** 2).mean()
    print("The bootstrap {}'s error rate is {:.3f}% when seed = {}.".format(i + 1, 100 * mse, seeds[i]))
    mse_list.append(mse)
    
avg_mse = np.array(mse_list).mean()
end_time = timeit.default_timer()

print("Average Error Rate =", avg_mse)
print("The computation time is about {} seconds.".format(end_time - start_time))

The bootstrap 1's error rate is 7.246% when seed = 798.
The bootstrap 2's error rate is 10.870% when seed = 344.
The bootstrap 3's error rate is 9.420% when seed = 739.
The bootstrap 4's error rate is 8.696% when seed = 271.
The bootstrap 5's error rate is 11.594% when seed = 847.
The bootstrap 6's error rate is 7.971% when seed = 292.
The bootstrap 7's error rate is 8.696% when seed = 812.
The bootstrap 8's error rate is 7.246% when seed = 512.
The bootstrap 9's error rate is 9.420% when seed = 272.
The bootstrap 10's error rate is 5.797% when seed = 939.
The bootstrap 11's error rate is 8.696% when seed = 243.
The bootstrap 12's error rate is 10.145% when seed = 475.
The bootstrap 13's error rate is 9.420% when seed = 569.
The bootstrap 14's error rate is 13.768% when seed = 661.
The bootstrap 15's error rate is 10.870% when seed = 126.
The bootstrap 16's error rate is 7.246% when seed = 831.
The bootstrap 17's error rate is 12.319% when seed = 904.
The bootstrap 18's error rate is 1

In [18]:
#b
from dask import compute, delayed
import dask.multiprocessing
import multiprocessing

multiprocessing.cpu_count()

4

In [17]:
start_timeb = timeit.default_timer()
def error_rate(boot_num, seeds, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.65, random_state = seeds[boot_num])
    logreg = LogisticRegression(solver="lbfgs", max_iter = 2000, n_jobs = None)
    results = logreg.fit(X_train, y_train)
    y_pred = results.predict(X_test)
    mse = ((y_test - y_pred) ** 2).mean()
    return mse

lazy_values = []
for i in range(100):
    lazy_values.append(delayed(error_rate)(i, seeds, X, y))
    
mse_list2 = compute(*lazy_values, scheduler = dask.multiprocessing.get, num_workers = 4)
end_timeb = timeit.default_timer()

for i in range(100):
    print("The bootstrap {}'s error rate is {:.3f}% when seed = {}.".format(i + 1, 100 * mse_list2[i], seeds[i]))

print("Average Error Rate =", np.array(mse_list2).mean())
print("The computation time is about {} seconds.".format(end_timeb - start_timeb))

The bootstrap 1's error rate is 7.246% when seed = 798.
The bootstrap 2's error rate is 10.870% when seed = 344.
The bootstrap 3's error rate is 9.420% when seed = 739.
The bootstrap 4's error rate is 8.696% when seed = 271.
The bootstrap 5's error rate is 11.594% when seed = 847.
The bootstrap 6's error rate is 7.971% when seed = 292.
The bootstrap 7's error rate is 8.696% when seed = 812.
The bootstrap 8's error rate is 7.246% when seed = 512.
The bootstrap 9's error rate is 9.420% when seed = 272.
The bootstrap 10's error rate is 5.797% when seed = 939.
The bootstrap 11's error rate is 8.696% when seed = 243.
The bootstrap 12's error rate is 10.145% when seed = 475.
The bootstrap 13's error rate is 9.420% when seed = 569.
The bootstrap 14's error rate is 13.768% when seed = 661.
The bootstrap 15's error rate is 10.870% when seed = 126.
The bootstrap 16's error rate is 7.246% when seed = 831.
The bootstrap 17's error rate is 12.319% when seed = 904.
The bootstrap 18's error rate is 1

As we can see from the results, the output is same as (b), but the computation time becomes much shorter.